# Captstone Project for IBM Data Science Professional Certificate

## The battle of the neighborhoods

### Introduction

Public Schools in Madrid had experimented a big increase of people demanding running activities for their children out of school premises. Madrid City Hall is creating a new project to cover those requests and, in order to better aim the activities to be done, Madrid City Hall asked for a way to group Public Schools taking into account the kind of activities that could take place near them since creating a program per school is not feasible due lack of resources.

### Objective

Create groups of Public Schools attending to the close amusement venues.

### Data Sources

Public School data (School name and coordinates): Madrid City Hall -  https://datos.madrid.es/egob/catalogo/202311-0-colegios-publicos.csv

Venues data (Category and coordinates): Foursquare API - https://foursquare.com/

### Metodology

These are the steps to create the groups of Public Schools:
+ Gather and clean Schools data 
+ Select coordinates per school and look for close venues of certain categories
+ Create a list of the most common kind of venues of those categories per school
+ Create clusters of schools attending to most common venues
+ Review information about generated clusters to better understand the schools categories

In [1]:
import pandas as pd
import numpy as np
import folium
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Gathering data from Madrid City Hall:

In [2]:
urlmaddata_csv = 'https://datos.madrid.es/egob/catalogo/202311-0-colegios-publicos.csv'
df_maddata = pd.read_csv(urlmaddata_csv,  encoding = "ISO-8859-1",sep=';')
df_maddata.set_index('PK')
df_maddata.head()

,PK,NOMBRE,DESCRIPCION-ENTIDAD,HORARIO,EQUIPAMIENTO,TRANSPORTE,DESCRIPCION,ACCESIBILIDAD,CONTENT-URL,NOMBRE-VIA,...,DISTRITO,COORDENADA-X,COORDENADA-Y,LATITUD,LONGITUD,TELEFONO,FAX,EMAIL,TIPO,Unnamed: 30
0,176850,Colegio Público Adolfo Suárez,NaN,NaN,Comedor - Horario ampliado - Planes de mejora.,BUS: 174,ENSEÑANZAS: Segundo ciclo de E. Infantil - Pri...,0,http://www.madrid.es/sites/v/index.jsp?vgnextc...,INFANTA CATALINA MICAELA,...,HORTALEZA,444807,4482402,40.488603,-3.652554,917 502 058 / 676 562 792,917 502 058,NaN,/contenido/entidadesYorganismos/CentrosEducaci...,
1,5075,Colegio Público Agustina Díez,NaN,NaN,Comedor - Horario ampliado,"METRO: Alto del Arenal BUS: 54 , 58 , 103 , 143",ENSEÑANZA: Educación infantil - Primaria,0,http://www.madrid.es/sites/v/index.jsp?vgnextc...,VESUBIO,...,PUENTE DE VALLECAS,445176,4471253,40.388191,-3.647237,917 770 073,917 770 073,NaN,/contenido/entidadesYorganismos/CentrosEducaci...,
2,4693138,Colegio Público Agustín Rodriguez Sahagún,NaN,NaN,Horario ampliado - Comedor - Planes de mejora,Bus: 142,ENSEÑANZA: Educacion Infantil - Primaria - Ens...,0,http://www.madrid.es/sites/v/index.jsp?vgnextc...,LA GRANJA DE SAN ILDEFONSO,...,VILLA DE VALLECAS,447288,4469000,40.368040,-3.622168,913 324 348,914 941 475,NaN,/contenido/entidadesYorganismos/CentrosEducaci...,
3,5076,Colegio Público Alberto Alcocer,NaN,NaN,Comedor - Transporte - Horario ampliado - Plan...,"METRO: Las Musas BUS: 38 , 48 , 140, 153",ENSEÑANZA: Educación Infantil - Primaria - Ens...,0,http://www.madrid.es/sites/v/index.jsp?vgnextc...,CANILLEJAS A VICALVARO,...,SAN BLAS-CANILLEJAS,448087,4476035,40.433338,-3.612052,913 202 609,913 200 446,NaN,/contenido/entidadesYorganismos/CentrosEducaci...,
4,5077,Colegio Público Alcalde de Móstoles,NaN,NaN,Transporte - Comedor - Horario ampliado - Plan...,"BUS: 34 , 39 , 139 , 117 , 17 ,",ENSEÑANZA: Educación Infantil - Primaria,0,http://www.madrid.es/sites/v/index.jsp?vgnextc...,ALDEANUEVA DE LA VERA,...,LATINA,434201,4470414,40.379837,-3.776448,917 055 465,915 095 129,NaN,/contenido/entidadesYorganismos/CentrosEducaci...,


Filter to keep only relevant information:

In [3]:
df_school_data = df_maddata[['PK', 'NOMBRE', 'DISTRITO','LATITUD','LONGITUD']]

df_school_data.head()

,PK,NOMBRE,DISTRITO,LATITUD,LONGITUD
0,176850,Colegio Público Adolfo Suárez,HORTALEZA,40.488603,-3.652554
1,5075,Colegio Público Agustina Díez,PUENTE DE VALLECAS,40.388191,-3.647237
2,4693138,Colegio Público Agustín Rodriguez Sahagún,VILLA DE VALLECAS,40.368040,-3.622168
3,5076,Colegio Público Alberto Alcocer,SAN BLAS-CANILLEJAS,40.433338,-3.612052
4,5077,Colegio Público Alcalde de Móstoles,LATINA,40.379837,-3.776448


Depicting the location of all schools

In [4]:
madrid_latitude=40.416775
madrid_longitude=-3.703790


# create map using latitude and longitude values
map_madrid = folium.Map(location=[madrid_latitude, madrid_longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, school_name in zip(df_school_data['LATITUD'], df_school_data['LONGITUD'], df_school_data['DISTRITO'], df_school_data['NOMBRE']):
    label = '{}, {}'.format(school_name, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  

    
map_madrid

Connecting to Foursquare API to get close venues of categories [1] "Outdoors & Recreation", "Museum", "Music Venue" and "Performing Arts Venue" for each school.

[1] https://developer.foursquare.com/docs/resources/categories


In [5]:
CLIENT_ID = 'KT2HVYWZLYN2KVZSWFUS0BOQVE54HQK2MAWLRRUVTCTQSEGM' 
CLIENT_SECRET = '5EFG3QXSXYSVR5KFQNVYTOE4DFHEOHYVYF5I12KUX2PCFNHS' 

VERSION = '20180605' # Foursquare API version

In [6]:
def getOutdoorsRecreationVenues(names, latitudes, longitudes, radius=500, LIMIT=50, category='4d4b7105d754a06377d81259,4bf58dd8d48988d181941735,4bf58dd8d48988d1e5931735,4bf58dd8d48988d1f2931735'):
    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            category)
            
        # make the GET request
        results = requests.get(url).json()["response"]["venues"]
                
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    nearOutdoorsRecreation_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearOutdoorsRecreation_venues.columns = ['School', 
                  'School Latitude', 
                  'School Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearOutdoorsRecreation_venues)






In [7]:
madrid_venues = getOutdoorsRecreationVenues(names=df_school_data['NOMBRE'],
                                   latitudes=df_school_data['LATITUD'],
                                   longitudes=df_school_data['LONGITUD']
                                  )

Colegio Público Adolfo Suárez
Colegio Público Agustina Díez
Colegio Público Agustín Rodriguez Sahagún
Colegio Público Alberto Alcocer
Colegio Público Alcalde de Móstoles
Colegio Público Alfonso X El Sabio
Colegio Público Alfredo di Stefano
Colegio Público Alhambra
Colegio Público Amadeo Vives
Colegio Público Amador de los Ríos
Colegio Público Amós Acero
Colegio Público Antonio Fontan
Colegio Público Antonio Machado
Colegio Público Antonio Moreno Rosales
Colegio Público Antonio de Nebrija
Colegio Público Aragón
Colegio Público Aravaca
Colegio Público Arcipreste de Hita
Colegio Público Arquitecto Gaudí
Colegio Público Asturias
Colegio Público Ausias March
Colegio Público Azorín
Colegio Público Barcelona
Colegio Público Blas de Lezo
Colegio Público Blas de Otero
Colegio Público Bolivia
Colegio Público Bravo Murillo
Colegio Público Breogán
Colegio Público Calderón de la Barca
Colegio Público Camilo José Cela
Colegio Público Capitán Cortés
Colegio Público Cardenal Herrera Oria
Colegio Públi

In [8]:
madrid_venues.head()

,School,School Latitude,School Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Colegio Público Adolfo Suárez,40.488603,-3.652554,Parque Sanchinarro,40.485844,-3.656469,Park
1,Colegio Público Adolfo Suárez,40.488603,-3.652554,Sanchinarro,40.491610,-3.656731,Neighborhood
2,Colegio Público Adolfo Suárez,40.488603,-3.652554,Valdefuentes,40.488199,-3.651074,Neighborhood
3,Colegio Público Adolfo Suárez,40.488603,-3.652554,Parque Infantil Pirata Pata de Lata,40.490574,-3.649876,Park
4,Colegio Público Adolfo Suárez,40.488603,-3.652554,Centro Cultural Sanchinarro,40.490163,-3.655297,Cultural Center


In [9]:
madrid_venues.shape

(5779, 7)

Creating a list of most common venues per school:

In [10]:
# one hot encoding
madrid_onehot = pd.get_dummies(madrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
madrid_onehot['School'] = madrid_venues['School'] 

# move neighborhood column to the first column
fixed_columns = [madrid_onehot.columns[-1]] + list(madrid_onehot.columns[:-1])
madrid_onehot = madrid_onehot[fixed_columns]

madrid_grouped = madrid_onehot.groupby('School').mean().reset_index()

madrid_grouped.head()

,School,Yoga Studio,Art Gallery,Art Museum,Athletics & Sports,Auditorium,Badminton Court,Bar,Basketball Court,Basketball Stadium,...,Toll Plaza,Town,Track,Track Stadium,Trail,Tree,Vineyard,Volcano,Volleyball Court,Well
0,Colegio Público Adolfo Suárez,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Colegio Público Agustina Díez,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Colegio Público Agustín Rodriguez Sahagún,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Colegio Público Alberto Alcocer,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Colegio Público Alcalde de Móstoles,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Removing some categories that are not interesting:

In [42]:
# Not interested in
not_interesting = ['Gym','Neighborhood','Fitness','Yoga','Bar','Dog','Restaurant','Club','Park','Plaza']

for no in not_interesting:
        cols=madrid_grouped.filter(like=no).columns  
        madrid_grouped = madrid_grouped.drop(columns=cols)

                                 
                                         
                                         
                                         

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['School']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
school_venues_sorted = pd.DataFrame(columns=columns)
school_venues_sorted['School'] = madrid_grouped['School']

for ind in np.arange(madrid_grouped.shape[0]):
    school_venues_sorted.iloc[ind, 1:] = return_most_common_venues(madrid_grouped.iloc[ind, :], num_top_venues)

school_venues_sorted.head()

,School,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Colegio Público Adolfo Suárez,Pool,Other Great Outdoors,Outdoors & Recreation
1,Colegio Público Agustina Díez,Pool,Garden,Other Great Outdoors
2,Colegio Público Agustín Rodriguez Sahagún,Dance Studio,Playground,Embassy / Consulate
3,Colegio Público Alberto Alcocer,Playground,Pool,Field
4,Colegio Público Alcalde de Móstoles,Dive Spot,River,Pool


Creating clusters of schools (after some tests, grouping in 3 kinds of schools seems to be a good approach )

In [68]:
# set number of clusters
kclusters = 3

madrid_grouped_clustering = madrid_grouped.drop('School', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(madrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_.shape

(244,)

In [69]:
madrid_grouped_clustering.shape

(244, 112)

In [70]:
#school_venues_sorted = school_venues_sorted.drop(columns=['Cluster Labels'])

In [71]:
# add clustering labels
school_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


ValueError: cannot insert Cluster Labels, already exists

In [72]:
madrid_merged = df_school_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
madrid_merged = madrid_merged.join(school_venues_sorted.set_index('School'), on='NOMBRE')

In [73]:
# Convert cluster from float to int
madrid_merged['Cluster Labels'] = madrid_merged['Cluster Labels'].astype(int)

# Drop rows with empty values
madrid_merged.dropna(subset=['Cluster Labels'], inplace=True)




Creating a map with Schools differentiating their group:

In [74]:
# create map
map_clusters = folium.Map(location=[madrid_latitude, madrid_longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(madrid_merged['LATITUD'], madrid_merged['LONGITUD'], madrid_merged['NOMBRE'], madrid_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       

       
map_clusters

Review most common venues per school group to better understand their definition

In [75]:
group0=madrid_merged.loc[madrid_merged['Cluster Labels'] == 0]
group0.head()


,PK,NOMBRE,DISTRITO,LATITUD,LONGITUD,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,176850,Colegio Público Adolfo Suárez,HORTALEZA,40.488603,-3.652554,0,Pool,Other Great Outdoors,Outdoors & Recreation
1,5075,Colegio Público Agustina Díez,PUENTE DE VALLECAS,40.388191,-3.647237,0,Pool,Garden,Other Great Outdoors
4,5077,Colegio Público Alcalde de Móstoles,LATINA,40.379837,-3.776448,0,Dive Spot,River,Pool
5,5078,Colegio Público Alfonso X El Sabio,VICALVARO,40.403201,-3.603695,0,Field,Pool,Other Great Outdoors
7,5079,Colegio Público Alhambra,FUENCARRAL-EL PARDO,40.483493,-3.726718,0,Pool,Soccer Field,Dance Studio


In [76]:
group0['1st Most Common Venue'].value_counts().head()


Pool                    40
Other Great Outdoors     9
Athletics & Sports       7
Soccer Field             2
Field                    2
Name: 1st Most Common Venue, dtype: int64

In [77]:

group0['2nd Most Common Venue'].value_counts().head()


Other Great Outdoors    16
Pool                    14
Playground              11
Field                    5
Soccer Field             4
Name: 2nd Most Common Venue, dtype: int64

In [78]:

group0['3rd Most Common Venue'].value_counts().head()

Playground              10
Other Great Outdoors    10
Pool                    10
Field                    5
Soccer Field             5
Name: 3rd Most Common Venue, dtype: int64

In [79]:
group1=madrid_merged.loc[madrid_merged['Cluster Labels'] == 1]
group1.head()

,PK,NOMBRE,DISTRITO,LATITUD,LONGITUD,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
3,5076,Colegio Público Alberto Alcocer,SAN BLAS-CANILLEJAS,40.433338,-3.612052,1,Playground,Pool,Field
6,10770379,Colegio Público Alfredo di Stefano,HORTALEZA,40.433338,-3.612052,1,Playground,Pool,Field
10,5083,Colegio Público Amós Acero,PUENTE DE VALLECAS,40.390223,-3.663775,1,Playground,Music Venue,Martial Arts Dojo
23,7635761,Colegio Público Blas de Lezo,FUENCARRAL-EL PARDO,40.509734,-3.670452,1,Playground,Pool,Tennis Court
26,5102,Colegio Público Bravo Murillo,FUENCARRAL-EL PARDO,40.480539,-3.718015,1,Playground,Soccer Field,Pool


In [80]:
group1['1st Most Common Venue'].value_counts().head()

Playground              38
Other Great Outdoors     3
Bridge                   1
Name: 1st Most Common Venue, dtype: int64

In [81]:
group1['2nd Most Common Venue'].value_counts().head()

Other Great Outdoors    15
Pool                     8
Playground               4
Field                    3
Soccer Field             2
Name: 2nd Most Common Venue, dtype: int64

In [82]:
group1['3rd Most Common Venue'].value_counts().head()

Music Venue           7
Soccer Field          4
Martial Arts Dojo     4
Field                 3
Athletics & Sports    3
Name: 3rd Most Common Venue, dtype: int64

In [83]:
group2=madrid_merged.loc[madrid_merged['Cluster Labels'] == 2]
group2.head()

,PK,NOMBRE,DISTRITO,LATITUD,LONGITUD,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
2,4693138,Colegio Público Agustín Rodriguez Sahagún,VILLA DE VALLECAS,40.368040,-3.622168,2,Dance Studio,Playground,Embassy / Consulate
9,5081,Colegio Público Amador de los Ríos,SALAMANCA,40.426280,-3.666558,2,Dance Studio,Theater,Museum
12,5087,Colegio Público Antonio Machado,CARABANCHEL,40.367425,-3.754198,2,Pool,Music School,Music Venue
13,5088,Colegio Público Antonio Moreno Rosales,CENTRO,40.411471,-3.701275,2,Theater,Other Great Outdoors,Performing Arts Venue
14,5085,Colegio Público Antonio de Nebrija,VILLAVERDE,40.351867,-3.686185,2,Other Great Outdoors,Playground,Music Venue


In [84]:
group2['1st Most Common Venue'].value_counts().head()

Other Great Outdoors    39
Theater                 14
Music Venue             14
Playground               8
Dance Studio             6
Name: 1st Most Common Venue, dtype: int64

In [85]:
group2['2nd Most Common Venue'].value_counts().head()

Other Great Outdoors     17
Music Venue              13
Playground               12
Performing Arts Venue     8
Theater                   6
Name: 2nd Most Common Venue, dtype: int64

In [86]:
group2['3rd Most Common Venue'].value_counts().head()

Music Venue             16
Other Great Outdoors     8
Pool                     8
Dance Studio             7
Playground               7
Name: 3rd Most Common Venue, dtype: int64

### Results

It's possible to have 3 differentiated groups of Public Schools in Madrid attending to their close venues than can be used for running activities with children:

+ Type 0: School with close water activities
+ Type 1: School with close sport centers and (possible) outdoor activities
+ Type 2: School with close music and arts venues

Regarding location of each type, there is a clear distinction between the distribution of type 2 and type 0/1. Type 2 schools are mainly located downtown while types 0 and 1 are in the suburbs. Type 1 is also mainly distributed among south-east of Madrid city. 



## Discussion

From the results it's clear that Madrid City Hall should promote art/music activities for Public Schools located in Downtown, since it this kind of venues are more easily to access from them. For Public Schools located in the suburbs the recommendation would be to promote sports activities, and it could be a good idea to specifically focus in water activities (pools) since those ones are close to the majority of Public Schools in that areas. 

Another option is to have two different programs for Public Schools in the suburbs, one focused in indoor/outdoor sports and another one focused just in pool activities.

## Conclusion

Even though we end up with three categories of Public Schools, due the kind of the most common categories of closest venues, it could make sense to have to big programs one focused in arts and music for downtown Public Schools and another one focused in sports for suburbs Public Schools.